In [1]:
import numpy as np
import pymc3 as pm

import aesara.tensor as at
from aesara.tensor.random.op import RandomVariable

from scipy import stats as st

import aesara

import arviz as az

%config InlineBackend.figure_format = "retina"
%matplotlib inline

aesara.__version__

You are running the v4 development version of PyMC3 which currently still lacks key features. You probably want to use the stable v3 instead which you can either install via conda or find on the v3 GitHub branch: https://github.com/pymc-devs/pymc3/tree/v3


'2.0.12'

In [34]:
K = 20; N = 50
M = 5; mu = 2.
rng = np.random.RandomState(seed=34)
y = rng.normal(loc=mu, scale=2., size=[N,])

In [3]:
def stick_breaking(betas):
    """
    betas ~ Beta(1, alpha)
    """
    sticks = at.concatenate(
        [
            [1],
            at.cumprod(1 - betas[:-1])
        ],
    )
    
    product = at.mul(betas, sticks)
    
    return at.concatenate(
        [
            product,
            [1 - at.sum(product)],
        ]
    )

In [57]:
with pm.Model() as model:
    
    concentration = pm.InverseGamma("concentration", alpha=1, beta=1)

    betas = pm.Beta("betas", 1., concentration + len(y), shape=(K,))
    weights = pm.Deterministic("weights", stick_breaking(betas))

    G0 = pm.Normal(name="G0", mu=0., sigma=3.)
    
    # idx only used for posterior sampling
    idx = pm.Bernoulli(name="post-mixture-idx", p=at.mul(concentration, at.inv(concentration + len(y))))

    cat_idx = pm.Categorical(name="cat-idx", p=np.ones_like(y)/len(y), shape=(K+1,))
    
    y_atoms = pm.Potential(
        name="atoms",
        var=lambda 
    )
    
    #     Gn = pm.Deterministic(
    #         name="Gn",
    #         var=G0[idx],
    #     )
    prior = pm.sample_prior_predictive(samples=1000,)

In [58]:
prior["weights"].mean(axis=0)

array([0.01781906, 0.0188505 , 0.01723841, 0.01735289, 0.01597581,
       0.01756036, 0.01639762, 0.01645864, 0.01592132, 0.01568358,
       0.01564189, 0.01474068, 0.01410327, 0.01428455, 0.01398772,
       0.75798372])

In [67]:
y[prior["cat-idx"][0]]

array([ 1.29587531,  4.52209386, -0.60050415,  1.53567575, -0.56298375,
        1.44354261,  3.67770284,  3.8512128 ,  1.29587531, -1.12235399,
        1.29587531, -0.92795379,  0.90935141,  0.70988939,  0.41779605,
        3.8512128 ])

In [ ]:
with pm.Model() as model:
    idx = pm.Categorical(name="cat", p=np.array([0.1, 0.2, 0.3, 0.4]))
    
    y_tensor = at.as_tensor_variable(y)
    y_atoms = pm.Deterministic(name="y_atoms", var=y_tensor[idx])
    
    prior = pm.sample_prior_predictive(samples=1000)